<a href="https://colab.research.google.com/github/ICBF-Analitica/taller_datos_abiertos_ICBF/blob/main/cuadernillos/ICBF_EstadisticasDescriptivas_TallerDatosAbiertosSept2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Grupo de Analítica Institucional ICBF](https://github.com/ICBF-Analitica/taller_datos_abiertos_ICBF/raw/main/assets/CabezoteModeloDatosAbiertos.png)

# 1. Instalación e importación de librerías

In [ ]:
# Instalación de librería necesaria para leer datos desde página web https://www.datos.gov.co/
!pip install sodapy

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from sodapy import Socrata

# 2. Lectura de conjunto de datos



---


#### Vamos a importar el conjunto de datos denominado "Caracterización de  Beneficiarios de las Modalidades de Prevención del ICBF", disponible en https://www.datos.gov.co/Inclusi-n-Social-y-Reconciliaci-n/Caracterizaci-n-de-Beneficiarios-de-las-Modalidade/g58z-k6f6. El identificador al final de la URL ("g58z-k6f6") es el que utilizaremos para leer los datos mediante la API de SODA

---







In [ ]:
# Se establece conexión con el portal de datos abiertos mediante la API de SODA. 
# Estableceremos una conexión anónima, ya que son datos públicos y por lo tanto no necesitamos autenticarnos
client = Socrata("www.datos.gov.co", None)

# Si quisieramos autenticarnos para descargar datos que son privados, utilizaríamos las siguientes líneas:
# client = Socrata(www.datos.gov.co,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# Con el método get_all() establecemos conexión específica con el conjunto de datos deseado.
# Con get_all() podemos leer todas las filas del conjunto de datos, que en este caso son 391.159, lo cual puede tomar algún tiempo.
# Como parámetro del método debemos ingresar el identificador del conjunto de datos "g58z"
results = client.get_all("g58z-k6f6")

# Si quisieramos leer solo una parte de los datos emplearíamos el método get() y como parámetro adicional el número de filas que queremos leer
# Ej: results = client.get("g58z-k6f6", limit=2000)

In [ ]:
# Ahora hacemos la lectura de los datos en un Dataframe de Pandas y revisamos su tamaño
df = pd.DataFrame.from_records(results)
print("El conjunto de datos tiene el siguiente número de (filas, columnas) = ", df.shape)
print("El tipo de objeto con el que vamos a trabajar es un ", type(df))

El conjunto de datos tiene el siguiente número de (filas, columnas) =  (391159, 19)
El tipo de objeto con el que vamos a trabajar es un  <class 'pandas.core.frame.DataFrame'>


In [ ]:
# Hacemos una exploración inicial de los datos al mostrar en pantalla los últimos 5 registros con el método tail() de Pandas
df.tail()

,a_o,codigo_departamento_atenci,area_misional,nombre_servicio,rango_edad,sexo,zona_ubicacion_beneficiario,agrupaci_n_etnica,grupo_etnico,presenta_discapacidad,v_ctima,nacionalidad,tipo_beneficiario_homologado,estado_atencion_beneficiario,corte,beneficiarios,departamento_atenci_n,municipio_atenci_n,codigo_municipio_atenci_n
391154,2020,70,FAMILIAS Y COMUNIDADES,MI FAMILIA URBANA,0 - 5 años,Hombre,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,25,Sucre,El Roble,70233
391155,2020,81,FAMILIAS Y COMUNIDADES,MI FAMILIA RURAL,0 - 5 años,Mujer,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,3,Arauca,Tame,81794
391156,2019,17,FAMILIAS Y COMUNIDADES,FAMILIAS CON BIENESTAR PARA LA PAZ,14 - 17 Años,Hombre,Cabecera,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,SI,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,201912,1,Caldas,Marquetalia,17444
391157,2018,5,FAMILIAS Y COMUNIDADES,COMUNIDADES RURALES,0 - 5 años,Mujer,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NO,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,201812,1,Antioquia,Yarumal,5887
391158,2019,73,FAMILIAS Y COMUNIDADES,FAMILIAS CON BIENESTAR PARA LA PAZ,9 - 13 Años,Hombre,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,SI,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,201912,3,Tolima,Ataco,73067


In [ ]:
# Verifiquemos qué tipos de datos se leyeron. Los datos tipo "object" son texto, es decir, variables cualitativas
df.dtypes

a_o                             object
codigo_departamento_atenci      object
area_misional                   object
nombre_servicio                 object
rango_edad                      object
sexo                            object
zona_ubicacion_beneficiario     object
agrupaci_n_etnica               object
grupo_etnico                    object
presenta_discapacidad           object
v_ctima                         object
nacionalidad                    object
tipo_beneficiario_homologado    object
estado_atencion_beneficiario    object
corte                           object
beneficiarios                   object
departamento_atenci_n           object
municipio_atenci_n              object
codigo_municipio_atenci_n       object
dtype: object

In [ ]:
# El número de beneficiarios, que está en la columna llamada "beneficiarios" si parece ser una variable cuantitativa discreta, por lo que debemos declararla como tal en el dataframe.
# Para ello declaramos dicha columna como tipo "int32", es decir, numérica entera/discreta
df["beneficiarios"] = df["beneficiarios"].astype('int32')

# Luego verificamos que el cambio se haya hecho
df.dtypes

a_o                             object
codigo_departamento_atenci      object
area_misional                   object
nombre_servicio                 object
rango_edad                      object
sexo                            object
zona_ubicacion_beneficiario     object
agrupaci_n_etnica               object
grupo_etnico                    object
presenta_discapacidad           object
v_ctima                         object
nacionalidad                    object
tipo_beneficiario_homologado    object
estado_atencion_beneficiario    object
corte                           object
beneficiarios                    int32
departamento_atenci_n           object
municipio_atenci_n              object
codigo_municipio_atenci_n       object
dtype: object

# 3. Entendimiento de los datos
#### De la celda anterior observamos que los beneficiarios están agrupados de acuerdo a sus características, y que el número de beneficiarios que comparten las mismas características se identifica mediante la columna "beneficiarios". Por lo tanto, para cualquier conteo que realicemos debemos tener en cuenta esta columna. 

#### Para empezar, observemos la siguiente tabla con estadísticas descriptivas

In [ ]:
# Con el método describe() de Pandas podemos obtener rápidamente algunas estadísticas descriptivas
df.describe(include="all")

,a_o,codigo_departamento_atenci,area_misional,nombre_servicio,rango_edad,sexo,zona_ubicacion_beneficiario,agrupaci_n_etnica,grupo_etnico,presenta_discapacidad,v_ctima,nacionalidad,tipo_beneficiario_homologado,estado_atencion_beneficiario,corte,beneficiarios,departamento_atenci_n,municipio_atenci_n,codigo_municipio_atenci_n
count,391159,391159,391159,391159,391159,391159,391159,391159,391159,391159,307505,391159,391159,391159,391159,391159.000000,391152,391152,391081
unique,3,36,4,62,8,5,3,5,8,2,2,3,9,2,3,NaN,35,1031,1264
top,2019,52,PRIMERA INFANCIA,DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRI...,6 Meses - 5 años,Mujer,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NO,Colombiano,NIÑO O NIÑA ENTRE 6 MESES Y 5 AÑOS Y 11 MESES,Activo,201912,NaN,Antioquia,Bogota. D.C.,11001
freq,155277,23200,214400,40800,150820,221113,201198,295494,295494,375987,202058,348137,155836,198090,155277,NaN,40640,2712,2712
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.082872,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.313936,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN


In [ ]:
# De la tabla anterior podemos observar que es necesario utilizar la columna "beneficiarios" para tener un conteo verdadero del número de beneficiarios.
# Calculemos el número total de beneficiarios que hay en el conjunto de datos
num_ben_total = df["beneficiarios"].sum()
print("Número total de beneficiarios en conjunto de datos: {:,} \n".format(num_ben_total))

# Observemos cuántos beneficiarios hay por año y por fecha de corte. Para esto, hacemos una tabla dinámica, como en Excel, con el método pivot_table() de Pandas
freq_ben_agno = pd.pivot_table(df, values='beneficiarios', index=['a_o', 'corte'], aggfunc=np.sum)
print("Beneficiarios por año y fecha de corte: \n \n", freq_ben_agno)

Número total de beneficiarios en conjunto de datos: 8,246,755 

Beneficiarios por año y fecha de corte: 
 
              beneficiarios
a_o  corte                
2018 201812        2950356
2019 201912        2743769
2020 202012        2552630


In [ ]:
# Creemos un dataframe solamente con el corte 2020.
# Para ello seleccionamos los datos que en la columna "a_o" tienen el valor de 2020. Lo hacemos mediante el comando .loc de Pandas
df_2020 = df.loc[df['a_o']=="2020"]

# Observemos cuántas filas tiene este dataframe y cuántos beneficiarios
print("El conjunto de datos tiene el siguiente número de (filas, columnas) = ", df_2020.shape)
num_ben_2020 = df_2020["beneficiarios"].sum()
print("Número total de beneficiarios en conjunto de datos: {:,} \n".format(num_ben_2020))
df_2020.tail()

El conjunto de datos tiene el siguiente número de (filas, columnas) =  (83654, 19)
Número total de beneficiarios en conjunto de datos: 2,552,630 



,a_o,codigo_departamento_atenci,area_misional,nombre_servicio,rango_edad,sexo,zona_ubicacion_beneficiario,agrupaci_n_etnica,grupo_etnico,presenta_discapacidad,v_ctima,nacionalidad,tipo_beneficiario_homologado,estado_atencion_beneficiario,corte,beneficiarios,departamento_atenci_n,municipio_atenci_n,codigo_municipio_atenci_n
391145,2020,25,FAMILIAS Y COMUNIDADES,MI FAMILIA URBANA,14 - 17 Años,Mujer,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NaN,Venezolano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,1,Cundinamarca,Villa De San Diego De Ubate,25843
391146,2020,52,FAMILIAS Y COMUNIDADES,MI FAMILIA URBANA,9 - 13 Años,Hombre,Resto,NARP,Afrocolombiano (a),NO,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,22,Nariño,San Andres De Tumaco,52835
391149,2020,99,FAMILIAS Y COMUNIDADES,MI FAMILIA URBANA,14 - 17 Años,Hombre,Cabecera,Indígena,Indígena,SI,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,1,Vichada,Puerto Carreño,99001
391154,2020,70,FAMILIAS Y COMUNIDADES,MI FAMILIA URBANA,0 - 5 años,Hombre,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,25,Sucre,El Roble,70233
391155,2020,81,FAMILIAS Y COMUNIDADES,MI FAMILIA RURAL,0 - 5 años,Mujer,Resto,No se autoreconoce,No se autoreconoce en ninguno de los anteriores,NO,NaN,Colombiano,NIÑOS. NIÑAS. ADOLESCENTES O ADULTOS,Activo,202012,3,Arauca,Tame,81794


In [ ]:
# Realicemos una frecuencia absoluta de la variable "sexo"

# Tabla dinámica que en las filas use la columna "sexo" y realice la suma de los valores de la columna "beneficiarios"
freqabs_sexo_2020 = pd.pivot_table(df_2020, index=['sexo'], values='beneficiarios', aggfunc=np.sum, margins=True, margins_name="Total")

# Cambiamos el nombre de la columna
freqabs_sexo_2020 = freqabs_sexo_2020.rename(columns={"beneficiarios": "Frecuencia absoluta"})

print("Frecuencia absoluta de beneficiarios en el 2020 por sexo: \n \n", freqabs_sexo_2020)

Frecuencia absoluta de beneficiarios en el 2020 por sexo: 
 
           Frecuencia absoluta
sexo                         
Hombre                1179722
Intersex                  272
Mujer                 1372636
Total                 2552630


In [ ]:
# Ahora una una frecuencia relativa de la variable "sexo"

# Reutilizamos el dataframe de la celda anterior. La frecuencia relativa es igual a la frecuencia absoluta dividida entre el número de observaciones
freqrel_sexo_2020 = (freqabs_sexo_2020 / num_ben_2020) * 100

# Cambiamos el nombre de la columna
freqrel_sexo_2020 = freqrel_sexo_2020.rename(columns={"Frecuencia absoluta": "Frecuencia relativa"})

print("Frecuencia relativa de beneficiarios en el 2020 por sexo: \n \n", freqrel_sexo_2020)

Frecuencia relativa de beneficiarios en el 2020 por sexo: 
 
           Frecuencia relativa
sexo                         
Hombre              46.215942
Intersex             0.010656
Mujer               53.773402
Total              100.000000


In [ ]:
# Unamos los dos dataframe anteriores para mostrar los datos en una sola table
freq_sexo_2020 = freqabs_sexo_2020.merge(freqrel_sexo_2020, left_index=True, right_index=True)
freq_sexo_2020

,Frecuencia absoluta,Frecuencia relativa
sexo,,
Hombre,1179722,46.215942
Intersex,272,0.010656
Mujer,1372636,53.773402
Total,2552630,100.000000


In [ ]:
# Realicemos el mismo proceso pero ahora exploremos la variable cualitativa "rango_edad"

#Frecuencia absoluta
freqabs_edad_2020 = pd.pivot_table(df_2020, index=['rango_edad'], values='beneficiarios', aggfunc=np.sum, margins=True, margins_name="Total")
# Cambiamos el nombre de la columna
freqabs_edad_2020 = freqabs_edad_2020.rename(columns={"beneficiarios": "Frecuencia absoluta"})

# Frecuencia relativa
freqrel_edad_2020 = (freqabs_edad_2020 / num_ben_2020) * 100

# Cambiamos el nombre de la columna
freqrel_edad_2020 = freqrel_edad_2020.rename(columns={"Frecuencia absoluta": "Frecuencia relativa"})

# Unimos los dos dataframe
freq_edad_2020 = freqabs_edad_2020.merge(freqrel_edad_2020, left_index=True, right_index=True)
freq_edad_2020

# ¿Qué hay por corregir en la siguiente tabla?

,Frecuencia absoluta,Frecuencia relativa
rango_edad,,
0 - 5 años,31506,1.234256
0-6 Meses,209153,8.193628
14 - 17 Años,100623,3.941934
6 - 8 Años,89879,3.521035
6 Meses - 5 años,1641050,64.288596
9 - 13 Años,160503,6.287750
Mayor a 18 Años,319915,12.532760
Sin información,1,0.000039
Total,2552630,100.000000




---

#### Hasta el momento solo hemos visto estadísticas descriptivas de variables cualitativas. Veamos ahora alguna variable cuantitativa. Para ello, creemos un dataframe que tenga el número de beneficiarios por municipio en 2020

In [ ]:
# Crear dataframe de número de beneficiarios por municipio en 2020
beneficiariosxmuni_2020 = pd.pivot_table(df_2020, index=["codigo_municipio_atenci_n", "municipio_atenci_n"], values='beneficiarios', aggfunc=np.sum).reset_index()
beneficiariosxmuni_2020

,codigo_municipio_atenci_n,municipio_atenci_n,beneficiarios
0,05001,Medellin,90770
1,05002,Abejorral,967
2,05004,Abriaqui,150
3,05021,Alejandria,329
4,05030,Amaga,1787
...,...,...,...
1109,97889,Yavarate,60
1110,99001,Puerto Carreño,5672
1111,99524,La Primavera,1023
1112,99624,Santa Rosalia,332


In [ ]:
# Podemos aumentar la información que tenemos incluyendo datos a nivel municipal desde otras fuentes.
# En este ejemplo, se agregará información de población municipal en 2020 desde un archivo .xlsx que se encuentra en el repositorio de Github

# Leemos el archivo en formato .xlsx desde el archivo disponible en el repositorio de GitHub. Se añade parámetros de nombre de la hoja que contiene la información y converter para que lea la columna de código de municipio como texto (string o object)
url = "https://github.com/ICBF-Analitica/taller_datos_abiertos_ICBF/raw/main/datos_adicionales/poblacion2020.xlsx"
datos_municipales = pd.read_excel(url, sheet_name="Final", converters={"DPMP": str})
datos_municipales

,DPMP,CabeceraMunicipal_2021,CentrosPoblados _RuralDisperso_2021,PobTotal_2021,PorcentajeRural_2021
0,05001,2490164,43260,2533424,0.017076
1,05002,7910,12377,20287,0.610095
2,05004,847,1888,2735,0.690311
3,05021,2637,2061,4698,0.438697
4,05030,15246,16037,31283,0.512643
...,...,...,...,...,...
1117,97889,0,1097,1097,1.000000
1118,99001,15697,4597,20294,0.226520
1119,99524,5996,3612,9608,0.375937
1120,99624,2643,1550,4193,0.369664


In [ ]:
# Unimos el archivo de beneficiarios por municipio en 2020 con estos nuevos datos municipales de población
beneficiariosxmuni_2020 = beneficiariosxmuni_2020.merge(datos_municipales, how="left", left_on="codigo_municipio_atenci_n", right_on="DPMP")
beneficiariosxmuni_2020

,codigo_municipio_atenci_n,municipio_atenci_n,beneficiarios,DPMP,CabeceraMunicipal_2021,CentrosPoblados _RuralDisperso_2021,PobTotal_2021,PorcentajeRural_2021
0,05001,Medellin,90770,05001,2490164,43260,2533424,0.017076
1,05002,Abejorral,967,05002,7910,12377,20287,0.610095
2,05004,Abriaqui,150,05004,847,1888,2735,0.690311
3,05021,Alejandria,329,05021,2637,2061,4698,0.438697
4,05030,Amaga,1787,05030,15246,16037,31283,0.512643
...,...,...,...,...,...,...,...,...
1109,97889,Yavarate,60,97889,0,1097,1097,1.000000
1110,99001,Puerto Carreño,5672,99001,15697,4597,20294,0.226520
1111,99524,La Primavera,1023,99524,5996,3612,9608,0.375937
1112,99624,Santa Rosalia,332,99624,2643,1550,4193,0.369664


In [ ]:
# Obtenemos las estadísticas descriptivas del dataframe
beneficiariosxmuni_2020.describe()

,beneficiarios,CabeceraMunicipal_2021,CentrosPoblados _RuralDisperso_2021,PobTotal_2021,PorcentajeRural_2021
count,1114.00000,1.114000e+03,1114.000000,1.114000e+03,1114.000000
mean,2291.40395,3.437517e+04,10834.064632,4.520923e+04,0.553935
std,8363.24755,2.631433e+05,13974.823315,2.659058e+05,0.238747
min,11.00000,0.000000e+00,0.000000,0.000000e+00,0.000474
25%,339.25000,1.998000e+03,3707.000000,6.781000e+03,0.391012
50%,879.00000,4.817500e+03,6785.000000,1.313350e+04,0.587502
75%,1972.75000,1.299125e+04,12669.750000,2.808950e+04,0.737553
max,214335.00000,7.715778e+06,181095.000000,7.743955e+06,1.000000
